In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,GRU
from tensorflow.python.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import initializers, regularizers, constraints, optimizers, layers
import re
tf.keras.backend.clear_session()

In [0]:
data=pd.read_csv('train_dataset.csv')

In [3]:
data.head()

,ID,Essayset,min_score,max_score,score_1,score_2,score_3,score_4,score_5,clarity,coherent,EssayText
0,1,1.0,0,3,1,1,1.0,1.0,1.0,average,worst,Some additional information that we would need...
1,2,1.0,0,3,1,1,NaN,1.5,1.0,excellent,worst,"After reading the expirement, I realized that ..."
2,3,1.0,0,3,1,1,1.0,1.0,1.5,worst,above_average,"What you need is more trials, a control set up..."
3,4,1.0,0,3,0,0,0.0,0.0,1.0,worst,worst,The student should list what rock is better an...
4,5,1.0,0,3,2,2,2.0,2.5,1.0,above_average,worst,For the students to be able to make a replicat...


In [4]:
data.isnull().sum()

ID             0
Essayset     157
min_score      0
max_score      0
score_1        0
score_2        0
score_3      147
score_4      136
score_5      144
clarity      138
coherent     145
EssayText      0
dtype: int64

In [0]:
data=data.dropna()

In [0]:
#data=data.fillna('0')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16176 entries, 0 to 17042
Data columns (total 12 columns):
ID           16176 non-null int64
Essayset     16176 non-null float64
min_score    16176 non-null int64
max_score    16176 non-null int64
score_1      16176 non-null int64
score_2      16176 non-null int64
score_3      16176 non-null float64
score_4      16176 non-null float64
score_5      16176 non-null float64
clarity      16176 non-null object
coherent     16176 non-null object
EssayText    16176 non-null object
dtypes: float64(4), int64(5), object(3)
memory usage: 1.6+ MB


In [0]:
X=data[['Essayset','EssayText']]

In [0]:
data['AVG']=data[['score_1','score_2','score_3','score_4','score_5']].mean(axis=1)

In [10]:
data.head()

,ID,Essayset,min_score,max_score,score_1,score_2,score_3,score_4,score_5,clarity,coherent,EssayText,AVG
0,1,1.0,0,3,1,1,1.0,1.0,1.0,average,worst,Some additional information that we would need...,1.0
2,3,1.0,0,3,1,1,1.0,1.0,1.5,worst,above_average,"What you need is more trials, a control set up...",1.1
3,4,1.0,0,3,0,0,0.0,0.0,1.0,worst,worst,The student should list what rock is better an...,0.2
4,5,1.0,0,3,2,2,2.0,2.5,1.0,above_average,worst,For the students to be able to make a replicat...,1.9
5,6,1.0,0,3,1,0,0.0,0.0,0.0,worst,worst,I would need the information of why you would ...,0.2


In [0]:
#data['NORMAVG']=[row['AVG']/3 if row['Essayset'] in([1,2,5,6]) else row['AVG']/2 for index, row in data.iterrows()]

In [0]:
#[row['AVG']/3 if row['Essayset'] in([1,2,5,6]) else row['AVG']/2 for index, row in data.iterrows()]

In [0]:
data1=data[data.Essayset.isin([1,2,5,6])]

In [0]:
X1=data1[['Essayset','EssayText']]

In [0]:
Y1=round(data1[['AVG']] )

In [0]:
data2=data[~data.Essayset.isin([1,2,5,6])]

In [0]:
X2=data2[['Essayset','EssayText']]

In [0]:
Y2=round(data2[['AVG']])

In [19]:
Y2.AVG.unique()

array([1., 2., 0.])

In [20]:
X.EssayText.str.len().max()

1819

In [21]:
#Subtopic column
input_2 = tf.keras.layers.Input(shape=(1,))
embed_2 = tf.keras.layers.Embedding(input_dim=4, 
                                    output_dim=10, 
                                    input_length=1
                                   )(input_2)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#Subtopic column
input_3 = tf.keras.layers.Input(shape=(1,))
embed_3 = tf.keras.layers.Embedding(input_dim=6, 
                                    output_dim=10, 
                                    input_length=1
                                   )(input_3)

In [0]:
embed_2=tf.keras.layers.Flatten()(embed_2)

In [0]:
embed_3=tf.keras.layers.Flatten()(embed_3)

In [0]:
#Vocab size
top_words = 1000

t = Tokenizer(num_words=top_words)
t.fit_on_texts(X1.EssayText.tolist())

#Get the word index for each of the word in the review
desc1 = t.texts_to_sequences(X1.EssayText.tolist())

In [0]:
from tensorflow.python.keras.preprocessing import sequence

#Each review size
max_review_length = 2000

desc1 = sequence.pad_sequences(desc1,maxlen=max_review_length,padding='post')

In [0]:
#Vocab size
top_words = 1000

t = Tokenizer(num_words=top_words)
t.fit_on_texts(X2.EssayText.tolist())

#Get the word index for each of the word in the review
desc2 = t.texts_to_sequences(X2.EssayText.tolist())

In [0]:
from tensorflow.python.keras.preprocessing import sequence

#Each review size
max_review_length = 2000

desc2 = sequence.pad_sequences(desc2,maxlen=max_review_length,padding='post')

In [0]:
input_12 = tf.keras.layers.Input(shape=(max_review_length,))
embed_12 = tf.keras.layers.Embedding(top_words, 100,   input_length=max_review_length)(input_12)
#embed_12 = GRU(128, dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=regularizers.l2(0.01))(embed_12)

In [0]:
embedding=tf.keras.layers.Flatten()(embed_12)

In [0]:
x = tf.keras.layers.concatenate([ embed_2,embedding])

In [32]:
x

<tf.Tensor 'concatenate/concat:0' shape=(?, 200010) dtype=float32>

In [0]:
x= tf.keras.layers.BatchNormalization()(x)




x = tf.keras.layers.Dense(30, activation='relu')(x)
x= tf.keras.layers.BatchNormalization()(x)

In [0]:
# Add output 
model_output = tf.keras.layers.Dense(4, activation='softmax')(x)

In [0]:
model = tf.keras.Model(inputs=[input_2,input_12], outputs=model_output)

In [0]:
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03, momentum=0.9, nesterov=True)



In [0]:
#Compile the model

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [38]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 2000)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 10)        40          input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 2000, 100)    100000      input_3[0][0]                    
__________________________________________________________________________________________________
flatten (F

In [0]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10)

In [0]:
Y1=tf.keras.utils.to_categorical(Y1, num_classes=4)

In [41]:
Y1

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [42]:
model.fit([X1[['Essayset']],desc1],
          Y1, 
          validation_split=0.2,
          epochs=10, 
          batch_size=100,callbacks=[callbacks])

Train on 4979 samples, validate on 1245 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
4979/4979 [==============================] - 2s 405us/sample - loss: 1.3114 - acc: 0.5156 - val_loss: 0.6382 - val_acc: 0.8410
Epoch 2/10
4979/4979 [==============================] - 1s 283us/sample - loss: 0.8192 - acc: 0.6515 - val_loss: 2.3274 - val_acc: 0.0273
Epoch 3/10
4979/4979 [==============================] - 1s 281us/sample - loss: 0.4846 - acc: 0.8291 - val_loss: 15.5015 - val_acc: 0.0273
Epoch 4/10
4979/4979 [==============================] - 1s 284us/sample - loss: 0.2392 - acc: 0.9488 - val_loss: 14.4094 - val_acc: 0.0940
Epoch 5/10
4979/4979 [==============================] - 1s 290us/sample - loss: 5.3794 - acc: 0.6252 - val_loss: 2.2416 - val_acc: 0.8410
Epoch 6/10
4979/4979 [==============================] - 1s 290us/sample - loss: 8.8782 - acc: 0.4433 - val_loss: 14.6049 - val_acc: 0.0940
Epoch 7/10
4979/4979 [==============================] - 1s 291us/sample -

In [0]:
x1 = tf.keras.layers.concatenate([ embed_3,embedding])

In [0]:
x1= tf.keras.layers.BatchNormalization()(x1)

x1 = tf.keras.layers.Dense(30, activation='relu')(x1)


In [0]:
Y2=tf.keras.utils.to_categorical(Y2, num_classes=3)

In [0]:
#Model 2
model_output1 = tf.keras.layers.Dense(3, activation='softmax')(x1)
model2 = tf.keras.Model(inputs=[input_3,input_12], outputs=model_output1)

In [0]:
#Compile the model

model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [48]:
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 2000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 10)        60          input_2[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 2000, 100)    100000      input_3[0][0]                    
__________________________________________________________________________________________________
flatten_1 

In [49]:
Y2

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [50]:
model2.fit([X2[['Essayset']],desc2],
          Y2, 
          validation_split=0.2,
          epochs=50, 
          batch_size=100,callbacks=[callbacks])

Train on 7961 samples, validate on 1991 samples
Epoch 1/50
7961/7961 [==============================] - 2s 314us/sample - loss: 0.8866 - acc: 0.5774 - val_loss: 1.1026 - val_acc: 0.4671
Epoch 2/50
7961/7961 [==============================] - 2s 277us/sample - loss: 0.8880 - acc: 0.7062 - val_loss: 1.1115 - val_acc: 0.4711
Epoch 3/50
7961/7961 [==============================] - 2s 278us/sample - loss: 0.4162 - acc: 0.8479 - val_loss: 1.4997 - val_acc: 0.4656
Epoch 4/50
7961/7961 [==============================] - 2s 281us/sample - loss: 0.3327 - acc: 0.8887 - val_loss: 1.3535 - val_acc: 0.4736
Epoch 5/50
7961/7961 [==============================] - 2s 282us/sample - loss: 0.4312 - acc: 0.9033 - val_loss: 1.4668 - val_acc: 0.4485
Epoch 6/50
7961/7961 [==============================] - 2s 279us/sample - loss: 0.1575 - acc: 0.9720 - val_loss: 1.4460 - val_acc: 0.4324
Epoch 7/50
7961/7961 [==============================] - 2s 286us/sample - loss: 0.1134 - acc: 0.9869 - val_loss: 1.8822 - va

In [0]:
test=pd.read_csv('test_dataset.csv')

In [52]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5224 entries, 0 to 5223
Data columns (total 7 columns):
ID           5224 non-null int64
Essayset     5224 non-null int64
min_score    5224 non-null int64
max_score    5224 non-null int64
clarity      5224 non-null object
coherent     5224 non-null object
EssayText    5224 non-null object
dtypes: int64(4), object(3)
memory usage: 285.8+ KB


In [0]:
test1=test[test.Essayset.isin([1,2,5,6])]
test2=test[~test.Essayset.isin([1,2,5,6])]

In [0]:
desc11 = t.texts_to_sequences(test1.EssayText.tolist())
desc12 = t.texts_to_sequences(test2.EssayText.tolist())

In [0]:
desc11 = sequence.pad_sequences(desc11,maxlen=max_review_length,padding='post')

In [0]:
desc12 = sequence.pad_sequences(desc12,maxlen=max_review_length,padding='post')

In [57]:
#Prediction
CATEGORIES = [0,1,2,3]
CATEGORIES2 = [0,1,2]

test2['essay_score']=[CATEGORIES2[i] for i in np.argmax(model2.predict([test2[['Essayset']],desc12]),axis=1)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [58]:
test1['essay_score']=[CATEGORIES[i] for i in np.argmax(model.predict([test1[['Essayset']],desc11]),axis=1)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
final=pd.concat([test1,test2],sort='ID')

In [0]:
#test['essay_score']=[row['essay_score']*3 if row['Essayset'] in([1,2,5,6]) else row['essay_score']*2 for index, row in test.iterrows()]

In [0]:
final=final.rename(index=str, columns={'ID': "id", 'Essayset': 'essay_set',})

In [0]:
final['essay_set']=round(final.essay_set)

In [0]:
final=final.sort_values(['id'])

In [0]:
final[['id','essay_set','essay_score']].to_csv('output_NLP.csv', index=False)